In [13]:
import pandas as pd

import pymongo
client = pymongo.MongoClient("localhost:27017")
db = client["London"]

In [14]:
londonneigh = db.get_collection("londonneigh")

In [15]:
#Function that iterates over the venues collections and the london neighborhoods collection containing polygons
# the function is used to find the number of venues in each neighborhood
# a df is created with the results of the crossing

In [16]:
def venues_london_df ():
    
    new_list = []
    venues = ["london_party", "london_schools","london_starbucks", "london_dogsaloon", "london_vegans"]
    neighborhoods = ["Camden", "Wesminster", "Kensington and Chelsea"]
    
    # 1. Loop every venue collection
    for venue in venues:
        venues_connection = db.get_collection(venue)
        
        # 2. Loop the neighborhoods
        for n in neighborhoods:
            londonneigh = db.get_collection("londonneigh")
            neighborhood_polygon = londonneigh.find_one({"properties.name": n})
            
            if neighborhood_polygon:
                results = list(venues_connection.find({
                    "location": {
                        "$geoWithin": {
                            "$geometry": neighborhood_polygon["geometry"]
                        }
                    }
                }))
                

                if results:
                    for i in results:
                        dict_ = {
                            "Venue name": i["name"],
                            "Category": venue,
                            "Lat": i["location"][1],
                            "Lon": i["location"][0],
                            "Neighborhood": n,
                        }

                        new_list.append(dict_)

    # Create a DataFrame from the result
    df = pd.DataFrame(new_list)

    return df

In [17]:
#Clean and analyse the df to extract conclusions

In [21]:
df = venues_london_df()
df = df.drop_duplicates()

In [26]:
df.to_csv("venues_london", sep=',', index=False, encoding='utf-8')

In [25]:
category_count = df.groupby(["Category", "Neighborhood"]).size().reset_index(name="Count")

# Pivot the table to get a proper format
category_count_table = category_count.pivot(index="Category", columns="Neighborhood", values="Count")
category_count_table = category_count_table.astype(int)

# Print the result
print(category_count_table)

Neighborhood      Camden  Kensington and Chelsea  Wesminster
Category                                                    
london_party          47                      35          32
london_schools        49                      44          46
london_starbucks       2                       1           2
london_vegans         15                      12          10


In [ ]:
#With the pivot table we observe the count of each venue in each neighborhood. 
#Due to excelling in all criterias, camden is chosen as the ideal neighborhood to locate the new offices